In [1]:
! ./download_covid_data

--2022-06-03 00:11:44--  https://rawcdn.githack.com/nytimes/covid-19-data/master/us-counties.csv
Resolving rawcdn.githack.com (rawcdn.githack.com)... 104.21.234.230, 104.21.234.231
Connecting to rawcdn.githack.com (rawcdn.githack.com)|104.21.234.230|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘us-counties.csv’

us-counties.csv         [          <=>       ]  99.94M  49.7MB/s    in 2.0s    

2022-06-03 00:11:56 (49.7 MB/s) - ‘us-counties.csv’ saved [104795654]



In [9]:
! ./download_county_populations

--2022-02-15 14:31:26--  https://raw.githack.com/sid-kap/housing-data-data/main/data/population/county/co-est2020-alldata.csv
Resolving raw.githack.com (raw.githack.com)... 2606:4700:3038::6815:eae6, 2606:4700:3038::6815:eae7, 104.21.234.230, ...
Connecting to raw.githack.com (raw.githack.com)|2606:4700:3038::6815:eae6|:443... connected.
HTTP request sent, awaiting response... 429 Too Many Requests
2022-02-15 14:31:27 ERROR 429: Too Many Requests.



In [2]:
import altair as alt
import pandas as pd 
from IPython.display import Markdown

pd.set_option('plotting.backend', 'altair')
pd.set_option('max_rows', 50)

# Load population data

In [3]:
county_populations = pd.read_csv('co-est2020-alldata.csv', encoding='latin_1')

In [4]:
county_populations_df = county_populations[[
    'CTYNAME',
    'STNAME',
    'POPESTIMATE2020',
]].rename(
    columns={
        'CTYNAME': 'county',
        'STNAME': 'state',
        'POPESTIMATE2020': 'population',
    }
).drop_duplicates()

county_populations_df['county'] = county_populations_df['county'].apply(
    lambda s: s.removesuffix(' County')
)

In [5]:
county_populations_df = pd.concat([
    county_populations_df,
    pd.DataFrame([{
        'county': 'New York City',
        'state': 'New York',
        'population': 8_804_190,
    }])
])

# Load covid data

In [6]:
df = pd.read_csv('./us-counties.csv')

In [7]:
df['date'] = pd.to_datetime(df['date'])

In [8]:
df_with_pop = df.merge(
    county_populations_df,
    on=['county', 'state'],
    how='left'
)

In [9]:
df_with_pop['cases_per_capita'] = df_with_pop['cases'] / df_with_pop['population']
df_with_pop['deaths_per_capita'] = df_with_pop['deaths'] / df_with_pop['population']

In [10]:
counties = {
    'Illinois': ['Cook'],
    'Washington': ['King'],
    'Oregon': ['Multnomah'],
    'Arizona': ['Maricopa'],
    'Texas': ['Collin', 'Dallas', 'Denton', 'Harris', 'Travis'],
    'California': ['San Francisco', 'Alameda', 'Contra Costa', 'Marin',
                  'San Mateo', 'Santa Clara', 'Los Angeles', 'Orange'],
    'District of Columbia': ['District of Columbia'],
    'New York': ['New York City', 'Westchester', 'Nassau', 'Suffolk', 'Rockland', 'Orange'],
    'New Jersey': ['Hudson', 'Bergen', 'Essex', 'Union'],
    'Massachusetts': ['Middlesex', 'Essex', 'Suffolk', 'Norfolk', 'Plymouth'],
}

In [11]:
def fix_zeros(series: pd.Series) -> pd.Series:
    num_zeros = 0
    new_values = []
    for value in series:
        if value == 0:
            num_zeros += 1
        else:
            num_days = num_zeros + 1
            new_values.extend([value / num_days] * num_days)
            num_zeros = 0
            
    new_values.extend([0] * num_zeros)
    return pd.Series(new_values, series.index)
        

for state, counties_list in counties.items():
    subset_df = df_with_pop[
        (df_with_pop['state'] == state)
        & df_with_pop['county'].isin(counties_list)
    ]
    
    daily_df = (
        subset_df
        .set_index(['date', 'county'])
        .drop(columns=['state', 'fips']).unstack().diff()
    )
    
    daily_df = daily_df.apply(fix_zeros)
    
    display(Markdown(f'# {state}'))
    display(
        (daily_df['cases_per_capita'] * 100_000)[-12:].round(1)
    )

# Illinois

county,Cook
date,
2022-05-02,26.3
2022-05-03,31.4
2022-05-04,53.0
2022-05-05,36.4
2022-05-06,71.6
2022-05-07,45.2
2022-05-08,45.2
2022-05-09,45.2
2022-05-10,41.9


# Washington

county,King
date,
2022-05-02,71.8
2022-05-03,-2.4
2022-05-04,-2.4
2022-05-05,49.2
2022-05-06,49.2
2022-05-07,40.2
2022-05-08,40.2
2022-05-09,40.2
2022-05-10,52.2


# Oregon

...

county,Nassau,New York City,Orange,Rockland,Suffolk,Westchester
date,,,,,,
2022-05-02,33.7,32.4,23.4,21.5,26.7,26.0
2022-05-03,36.3,27.5,28.8,30.7,28.1,29.1
2022-05-04,58.5,26.7,40.5,40.8,49.1,48.8
2022-05-05,67.0,45.0,57.4,42.6,50.3,53.1
2022-05-06,73.4,51.1,54.3,54.3,58.7,57.7
2022-05-07,66.7,4.1,56.1,40.2,56.3,56.9
2022-05-08,70.5,4.1,42.1,85.5,57.2,51.5
2022-05-09,42.0,149.6,28.3,25.4,36.2,39.2
2022-05-10,46.5,26.0,38.2,31.9,41.0,37.6


# New Jersey

county,Bergen,Essex,Hudson,Union
date,,,,
2022-05-02,41.0,44.5,20.2,28.3
2022-05-03,46.5,25.0,24.9,31.1
2022-05-04,58.4,34.9,35.9,39.6
2022-05-05,60.0,44.1,44.1,41.8
2022-05-06,64.6,47.0,51.7,44.5
2022-05-07,64.1,43.3,37.2,42.5
2022-05-08,54.5,48.2,25.2,34.2
2022-05-09,29.0,29.0,23.2,26.3
2022-05-10,73.8,43.7,41.4,45.4


# Massachusetts

county,Essex,Middlesex,Norfolk,Plymouth,Suffolk
date,,,,,
2022-05-02,27.6,36.2,25.5,18.7,34.9
2022-05-03,30.5,45.3,45.0,21.4,61.1
2022-05-04,34.0,47.9,40.7,33.6,56.6
2022-05-05,64.1,69.5,55.4,38.2,95.9
2022-05-06,42.8,61.7,49.2,37.6,77.5
2022-05-07,37.3,48.9,39.0,27.3,43.0
2022-05-08,37.3,48.9,39.0,27.3,43.0
2022-05-09,37.3,48.9,39.0,27.3,43.0
2022-05-10,42.3,39.4,46.1,39.3,54.0


In [12]:
def plot_for_state(df_with_pop, state, counties_list, lookback_days=30):
    subset_df = df_with_pop[
        (df_with_pop['state'] == state)
        & df_with_pop['county'].isin(counties_list)
    ]
    
    daily_df = (
        subset_df
        .set_index(['date', 'county'])
        .drop(columns=['state', 'fips']).unstack().diff()
    )
    
    daily_df = daily_df.apply(fix_zeros)
    
    # replace negative numbers with 0
    daily_df = daily_df.clip(lower=0)
    
    daily_df_melted = (
        (daily_df['cases_per_capita'][-lookback_days:] * 100_000).reset_index().rename_axis(None, axis='columns')
        .melt(id_vars=['date'], var_name='county', value_name='cases_per_capita_100k')
    )
    
    return alt.Chart(daily_df_melted).mark_line(clip=True).encode(
        x=alt.X('date', axis=alt.Axis(format='%Y-%m-%d', labelAngle=-60, tickCount=20, labelPadding=0.5, labelOverlap=True)),
        y=alt.Y(
            'cases_per_capita_100k', 
            axis=alt.Axis(title='Daily cases per 100k'), 
            scale=alt.Scale(domain=[0, 500]),
        ),
        color='county',
    )

In [13]:
for state, counties_list in counties.items():
    chart = plot_for_state(df_with_pop, state, counties_list, lookback_days=120)
    
    display(Markdown(f'# {state}'))
    display(chart)
    # display((daily_df['cases_per_capita'] * 100_000)[-12:].plot())

# Illinois

alt.Chart(...)

# Washington

alt.Chart(...)

# Oregon

...

alt.Chart(...)

# New Jersey

alt.Chart(...)

# Massachusetts

alt.Chart(...)

In [14]:
plot_for_state(df_with_pop, 'California', counties['California'], lookback_days=7)

alt.Chart(...)